<a href="https://colab.research.google.com/github/evmpython/CAT010_UNIFEI_2025/blob/main/CAT010_Aula_05_plota_GLM_Parte_1_20s_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


#  🟢 **Disciplina CAT010** - Aplicações dos Satélites em Ciências Atmosféricas


---

## **Aula Prática 5 - Parte 1:** $\underline{Plotando\ e\ Analisando\ Relâmpagos\ Estimados\ pelo\ Satélite\ GOES-16}$
---
**OBJETIVO:** Nesta aula prática aprenderemos como trabalhar com dados de relâmpagos estimado por satélite. Para isto usaremos os dados do sensor [Geostationary Lightning Mapper (GLM)](https://space.oscar.wmo.int/instruments/view/glm) abordo do satélite [Geostationary Operational Environmental Satellite - 16 (GOES-16)](https://space.oscar.wmo.int/satellites/view/goes_16). O objetivo dessa **Parte 1** é aprender como plotar e analisar os dados a cada 20s do GLM e combiná-los com imagens de satélite.

---


**DADOS DE ENTRADA**: Arquivos de relâmpagos.

1. $\underline{Dados\ de\ Relâmpagos}$: sensor GLM do satélite GOES-16
- **Tipo do dado:** pontuais
- **Formato do dado:** arquivo NETCDF
- **Nome do arquivo:** OR_GLM-L2-LCFA_G16_s20210010306000_e20210010306204_c20210010306215.nc
- **Fonte dos dados:** FTP da [Amazon](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/)

    ----------

2. $\underline{Dados\ de\ Tempertura\ de\ Brilho}$: sensor ABI do satélite GOES-16
- **Tipo do dado:** matriz
- **Formato do dado:** arquivo NETCDF
- **Nome do arquivo:** S10635346_202001231200.nc
- **Fonte dos dados:** FTP do [CPTEC/INPE](http://ftp.cptec.inpe.br/goes/goes16/retangular/ch13/)


---

**DADOS DE SAÍDA:** .
- **Tipo do dado:** Figura
- **Formato do dado:** arquivos JPG
- **Imagens geradas:**
    1. Fig_1_goes16_flash_20210101_1700.jpg
    2. Fig_2_goes16_flash_20210101_1700_recorte.jpg
    3. Fig_3_goes16_flash_20210101_1700_recorte.jpg
    4. Fig_3_animation.gif
    5. Fig_4_goes16_evento_grupo_flash_20210101_1700_recorte.jpg

---

**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos são realizados nesse código:
1.   Instalação de bibliotecas
2.   Download de arquivos auxiliares
3.   Monta o google drive
4.   Define o caminho do google drive
5.   Importação das bibliotecas
6.   Define funções
7.   Conhecendo os dados de relâmpagos a cada 20 s do sensor GLM
8.   Combinando imagem de satélite com flashes: América do Sul
9.   Combinando imagem de satélite com flashes: Área com recorte
10. Faz animação das imagens combinadas de satélite e flashes: Área com recorte
11. Combinando imagem de satélite com eventos, grupos e flashes: Área com recorte


---
**OBSERVAÇÕES IMPORTANTES**:
1. Fazer mudanças, veja em: **### >>> MUDANÇAS AQUI <<< ###**
    - 1. Caminho do seu drive

---
- **Realizado por:** Enrique V. Mattos - 18/05/2022
- **Atualizado por:** Enrique V. Mattos - 11/11/2024


---


# **1° Passo:** Instalando as bibliotecas

In [ ]:
!pip install -q ultraplot cartopy boto3 rasterio pyproj geopandas salem descartes boto3 gdal

# **2° Passo:** Download de Arquivos Auxiliares


In [ ]:
# Download do arquivo "utilities.py"
!wget -c https://github.com/evmpython/CAT010_UNIFEI_2025/raw/main/utils/utilities_goes16.py

# Download da paleta de cores para o canal do infravermelho
!wget -c https://github.com/evmpython/CAT010_UNIFEI_2025/raw/main/utils/ir.cpt

# **3° Passo:** Montando drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **4° Passo:** Caminho do diretório - **### >>> MUDANÇAS AQUI <<< ###**

- Insira aqui o caminho da pasta do seu Google Drive onde esta o código dessa aula: `alterar aqui`

In [ ]:
dir = '/content/drive/MyDrive/00_DISPONIBILIZAÇÃO_DE_AULAS/2_SEGUNDO_SEMESTRE/CAT010 - APLICAÇÕES DOS SATÉLITES EM CIÊNCIAS ATMOSFÉRICAS/AULAS/CAT010 - PYTHON PARA SATÉLITES METEOROLÓGICOS/AULA_05_RELAMPAGOS/'

- **Criando a pasta de saída ("output")**


In [ ]:
import os
os.makedirs(f'{dir}output', exist_ok=True)

# **5° Passo:** Importando as bibliotecas

In [ ]:
# ======================================================
# BIBLIOTECAS PARA PROCESSAMENTO E ANÁLISE DE DADOS
# ======================================================
import numpy as np                                      # Cálculos numéricos com arrays multidimensionais
import pandas as pd                                     # Manipulação e análise de dados em tabelas
import xarray as xr                                     # Trabalho com datasets multidimensionais (netCDF, HDF5)
import glob                                             # Listagem de arquivos por padrão de nome

# ======================================================
# BIBLIOTECAS PARA VISUALIZAÇÃO E MAPEAMENTO
# ======================================================
import ultraplot as uplt                                # Visualização avançada para dados meteorológicos
import cartopy.crs as ccrs                              # Sistemas de referência para projeções de mapas
import cartopy.io.shapereader as shpreader              # Leitura de arquivos shapefile
import salem                                            # Ferramentas geoespaciais integradas com xarray
from matplotlib.colors import LinearSegmentedColormap   # Criação de mapas de cores customizados
from matplotlib import cm                               # Utilitários para gerenciamento de cores em gráficos
import imageio                                          # Leitura e escrita de arquivos de imagem

# ======================================================
# BIBLIOTECAS PARA MANIPULAÇÃO DE TEMPO E DATAS
# ======================================================
import calendar                                         # Funções relacionadas a calendário
from datetime import timedelta, date, datetime          # Manipulação de datas e horários
import time as t                                        # Funções para medição e controle de tempo
from time import process_time                           # Medição precisa de tempo de processamento

# ======================================================
# BIBLIOTECAS PARA ACESSO A DADOS E ARMAZENAMENTO
# ======================================================
import os                                               # Interação com o sistema operacional
import boto3                                            # Interface com serviços AWS (Amazon Web Services)
from botocore import UNSIGNED                           # Configuração para acesso anônimo na AWS
from botocore.config import Config                      # Configurações do cliente AWS

# ======================================================
# UTILITÁRIOS PERSONALIZADOS
# ======================================================
from utilities_goes16 import loadCPT                     # Carregamento de paletas de cores para dados GOES-16

# ======================================================
# CONTROLE DE AVISOS
# ======================================================
import warnings                                         # Gerenciamento de mensagens de alerta
warnings.filterwarnings("ignore")                       # Supressão de avisos durante a execução

# **6° Passo:** Definindo funções

## - **Função que baixa os dados do GLM (GOES-16) no site de Amazon**

In [ ]:
def download_GLM(yyyymmddhhmnss: str, path_dest: str) -> str:

    """
    Faz o download de dados do sensor GLM (Geostationary Lightning Mapper) do satélite GOES-16.

    Os arquivos são buscados no repositório público da AWS (Amazon Web Services) e salvos
    localmente no formato NetCDF (.nc).

    Parâmetros
    ----------
    yyyymmddhhmnss : str
        Data no formato 'YYYYMMDDHHMMSS' para buscar os dados.
        Exemplo: '20230101120000' (1º de Janeiro de 2023, 12:00:00 UTC)
    path_dest : str
        Diretório de destino onde o arquivo será salvo.
        Exemplo: '/content/dados/GLM'

    Retorna
    -------
    str
        Nome do arquivo baixado (sem extensão) em caso de sucesso.
        Retorna -1 se nenhum arquivo for encontrado para a data especificada.

    Exemplos
    --------
    >>> download_GLM('20230101120000', '/content/dados/GLM')
    'OR_GLM-L2-LCFA_G16_s2023001120000'
    """

    # Cria o diretório de destino se não existir
    os.makedirs(path_dest, exist_ok=True)

    # Extrai componentes da data
    dt = datetime.strptime(yyyymmddhhmnss, '%Y%m%d%H%M%S')
    year = dt.strftime('%Y')
    day_of_year = dt.strftime('%j')  # Dia juliano (001-366)
    hour = dt.strftime('%H')
    minute = dt.strftime('%M')
    second = dt.strftime('%S')

    # Configuração do cliente AWS S3 (acesso público)
    bucket_name = 'noaa-goes16'
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

    # Estrutura do arquivo no repositório AWS
    product_name = "GLM-L2-LCFA"
    prefix = (f'{product_name}/{year}/{day_of_year}/{hour}/'
        f'OR_{product_name}_G16_s{year}{day_of_year}{hour}{minute}{second}')

    # Busca o arquivo no servidor S3
    s3_result = s3_client.list_objects_v2(Bucket=bucket_name,
                                          Prefix=prefix,
                                          Delimiter="/")

    # Verifica se existem resultados
    if 'Contents' not in s3_result:
        print(f'Nenhum arquivo encontrado para: {yyyymmddhhmnss}, Produto-{product_name}')
        return -1

    # Processa o arquivo encontrado
    for obj in s3_result['Contents']:
        key = obj['Key']

    # Extrai o nome do arquivo (sem extensão)
    file_name = key.split('/')[-1].split('.')[0]
    dest_path = f'{path_dest}/{file_name}.nc'

    # Verifica se o arquivo já existe localmente
    if os.path.exists(dest_path):
        print(f'Arquivo {dest_path} já existe')
    else:
        print(f'Baixando arquivo: {dest_path}')
        s3_client.download_file(bucket_name, key, dest_path)

    return file_name

## - **Função que baixa as imagens do ABI no ftp do CPTEC**

In [ ]:
def download_ABI_CPTEC(ano: str, mes: str, dia: str, hor: str, min: str, canal: str) -> str:

    """
    Faz o download de dados do satélite GOES-16 (ABI) do repositório do CPTEC/INPE.

    Os arquivos são baixados do FTP público do CPTEC no formato NetCDF (.nc) e salvos
    no diretório '/content/' do ambiente de execução.

    Parâmetros
    ----------
    ano : str
        Ano com 4 dígitos (ex: '2023')
    mes : str
        Mês com 2 dígitos (ex: '01' para Janeiro)
    dia : str
        Dia com 2 dígitos (ex: '15')
    hor : str
        Hora com 2 dígitos (ex: '12' para 12:00 UTC)
    min : str
        Minutos com 2 dígitos (ex: '00')
    canal : str
        Canal do ABI a ser baixado. Valores aceitos: 'ch02', 'ch08', 'ch13'

    Retorna
    -------
    str
        Caminho completo do arquivo baixado no formato:
        '/content/S10635334_YYYYMMDDHHMM.nc'

    Exemplos
    --------
    >>> download_ABI_CPTEC('2023', '01', '15', '12', '00', 'ch13')
    '/content/S10635346_202301151200.nc'

    Notas
    -----
    Os canais disponíveis correspondem a:
    - ch02: Visível (0.64 µm)
    - ch08: Infravermelho (6.2 µm)
    - ch13: Infravermelho (10.3 µm)
    """

    # Validação básica dos parâmetros
    if canal not in ['ch02', 'ch08', 'ch13']:
        raise ValueError("Canal inválido. Use 'ch02', 'ch08' ou 'ch13'")

    print(f'Baixando dado ===> DATA={ano}/{mes}/{dia} {hor}:{min} / CANAL={canal}')

    # Configuração do FTP do CPTEC
    ftp_cptec = 'http://ftp.cptec.inpe.br/goes/'

    # Mapeamento de canais para códigos de produto
    canal_para_codigo = {'ch02': 'S10635334',
                         'ch08': 'S10635340',
                         'ch13': 'S10635346'}
    codigo_produto = canal_para_codigo[canal]

    # Construção da URL do arquivo remoto
    url_arquivo = (f'{ftp_cptec}goes16/retangular/{canal}/{ano}/{mes}/'
                   f'{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc')

    # Nome do arquivo local
    nome_arquivo_local = f'{codigo_produto}_{ano}{mes}{dia}{hor}{min}.nc'
    caminho_local = f'/content/{nome_arquivo_local}'

    # Verifica se o arquivo já existe antes de baixar
    if not os.path.exists(caminho_local):

        # Download usando wget (com opção -c para continuar downloads interrompidos)
        !wget -c {url_arquivo}

    else:

        print(f'Arquivo {caminho_local} já existe. Pulando download.')

    return caminho_local

## - **Função que baixa e acumula flashes no intervalo temporal da imagem**

In [ ]:
def download_e_acumula_GLM(ano: str, mes: str, dia: str, hor: str, min: str) -> tuple:

    """
    Baixa e acumula dados de raios do sensor GLM (GOES-16) em um período de 10 minutos.

    Esta função realiza o download de múltiplos arquivos GLM em intervalos de 20 segundos,
    acumulando as posições de eventos, grupos e flashes de raios.

    Parâmetros
    ----------
    ano : str
        Ano com 4 dígitos (ex: '2023')
    mes : str
        Mês com 2 dígitos (ex: '01' para Janeiro)
    dia : str
        Dia com 2 dígitos (ex: '15')
    hor : str
        Hora com 2 dígitos (ex: '12' para 12:00 UTC)
    min : str
        Minutos com 2 dígitos (ex: '00')

    Retorna
    -------
    tuple
        Uma tupla contendo seis arrays numpy na seguinte ordem:
        - lats_event: Latitudes dos eventos de raios
        - lons_event: Longitudes dos eventos de raios
        - lats_group: Latitudes dos grupos de raios
        - lons_group: Longitudes dos grupos de raios
        - lats_flash: Latitudes dos flashes de raios
        - lons_flash: Longitudes dos flashes de raios

    Exemplo
    -------
    >>> lats_event, lons_event, lats_group, lons_group, lats_flash, lons_flash = \
    ...     download_e_acumula_GLM('2023', '01', '15', '12', '00')
    """

    # Converte para datetime e define o período de 10 minutos
    dt_ini = datetime(int(ano), int(mes), int(dia), int(hor), int(min))
    dt_end = dt_ini + timedelta(minutes=10)

    # Inicializa arrays de armazenamento
    lats_event, lons_event = np.array([]), np.array([])
    lats_group, lons_group = np.array([]), np.array([])
    lats_flash, lons_flash = np.array([]), np.array([])

    # Loop de acumulação
    current_time = dt_ini

    while current_time < dt_end:

        try:

            # Formata a data para o padrão do arquivo
            yyyymmddhhmnss = current_time.strftime('%Y%m%d%H%M%S')

            # Download do arquivo GLM
            file_glm = download_GLM(yyyymmddhhmnss, '/content')

            if file_glm != -1:  # Verifica se o download foi bem-sucedido

                # Leitura do arquivo NetCDF
                with xr.open_dataset(f'/content/{file_glm}.nc') as ds:

                    # Acumula as coordenadas
                    lats_event = np.append(lats_event, ds['event_lat'].values)
                    lons_event = np.append(lons_event, ds['event_lon'].values)

                    lats_group = np.append(lats_group, ds['group_lat'].values)
                    lons_group = np.append(lons_group, ds['group_lon'].values)

                    lats_flash = np.append(lats_flash, ds['flash_lat'].values)
                    lons_flash = np.append(lons_flash, ds['flash_lon'].values)

            # Incrementa 20 segundos
            current_time += timedelta(seconds=20)

        except Exception as e:
            print(f"Erro ao processar {current_time}: {str(e)}")
            current_time += timedelta(seconds=20)  # Continua mesmo com erro

    return lats_event, lons_event, lats_group, lons_group, lats_flash, lons_flash

# **PARTE 1)**: Conhecendo os dados de relâmpagos a cada **20 s** do sensor GLM

##**a) nome do arquivo**

---
1. **Descrição:** São os dados originais gerados pela NOAA.
---
2. **Onde baixar os dados:** Os dados de relâmpagos do sensor GLM abordo do satélite GOES-16 podem ser baixados no site da Amazon ou do CPTEC.
---
   1. **FORMA 1:** Os dados do GOES-16/17 são disponibilizados pela Amazon nesse [link](https://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?source=aws&satellite=noaa-goes16&domain=F&product=GLM-L2-LCFA&date=2021-03-04&hour=13) e nesse [repositório](https://noaa-goes16.s3.amazonaws.com/index.html#GLM-L2-LCFA/).

- Formato dos arquivos: Netcdf. Os dados estão como vetores.
- Como baixar pelo Colab: !wget https://noaa-goes16.s3.amazonaws.com/GLM-L2-LCFA/2021/302/01/OR_GLM-L2-LCFA_G16_s20213020100000_e20213020100200_c20213020100228.nc

- Exemplos dos arquivos:
    - OR_GLM-L2-LCFA_G16_s20210101300000_e20210101300204_c20210101300214.nc
    - OR_GLM-L2-LCFA_G16_s20210101300200_e20210101300404_c20210101300424.nc
    - OR_GLM-L2-LCFA_G16_s20210101300400_e20210101301005_c20210101301021.nc
    - OR_GLM-L2-LCFA_G16_s20210101301000_e20210101301205_c20210101301215.nc
    - OR_GLM-L2-LCFA_G16_s20210101301200_e20210101301405_c20210101301428.nc
    - OR_GLM-L2-LCFA_G16_s20210101301400_e20210101302005_c20210101302018.nc.

- Informações que constam nos nomes dos arquivos:  
    - **OR_GLM-L2-LCFA**: Nome do sensor (GLM), Nível de processamento (L2)
    - **G16**: Nome do satélite
    -**s20210101300000**: inicio do Scan (s). ano (2021), dia juliano (010), hora (13), minuto (00) e segundos (000)
    -**e20210101300204**: fim do Scan (s). ano (2021), dia juliano (010), hora (13), minuto (00) e segundos (204=20.4)
    -**c20210101300214**: data de criação do arquivo (c). ano (2021), dia juliano (010), hora (13), minuto (00) e segundos (214=21.4)


---
   2. **FORMA 2:** Os dados do GOES-16 também disponibilizados esses dados e podem serem baixados no [FTP](http://ftp.cptec.inpe.br/goes/goes16/glm/) do CPTEC. A explicação sobre os arquivos é a mesma dada para os dados da Amazon.
---

In [ ]:
# nome do arquivo
file_20s = 'OR_GLM-L2-LCFA_G16_s20210010307000_e20210010307203_c20210010307223.nc'

##**b) leitura do arquivo**

In [ ]:
# url do arquivo no github
url = f'https://github.com/evmpython/CAT010_UNIFEI_2025/raw/main/input/aula_05/{file_20s}'

# baixa o arquivo do github
!wget -c {url}

# leitura do arquivo
ds = xr.open_dataset(f'{file_20s}')

In [ ]:
# exibe os dados
ds

##**c) exibindo os dados**

In [ ]:
# latitude dos eventos
ds['event_lat']

In [ ]:
# longitude dos eventos
ds['event_lon']

In [ ]:
# transformando a latitude para um array
lats_event = np.array((ds['event_lat']))
lats_event

In [ ]:
# formato do array
lats_event.shape

In [ ]:
# transformando a latitude para um array
lons_event = np.array((ds['event_lon']))
lons_event

In [ ]:
# formato do array
lons_event.shape

In [ ]:
# latitude dos grupos
ds['group_lat']

In [ ]:
# longitude dos grupos
ds['group_lon']

In [ ]:
# latitude dos flashes
ds['flash_lat']

In [ ]:
# longitude dos flashes
ds['flash_lon']

# **PARTE 2)**: Combinando imagem de satélite com flashes: América do Sul

##**a) Definindo data/horário e canal da imagem**

In [ ]:
# define data/horário e canal
ano, mes, dia, hor, min, canal = '2021', '01', '01', '17', '00', 'ch13'

In [ ]:
# baixa imagem de satélite do FTP do CPTEC
file_ir_download = download_ABI_CPTEC(ano, mes, dia, hor, min, canal)

##**b) Acumula os flashes**

In [ ]:
# define data a INICIAL para acumular os relâmpagos
date_ini = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)))

# define data FINAL para acumular os relâmpagos
date_end = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)) + timedelta(minutes=10)) # soma 10 minutos na data inicial

In [ ]:
# mostra na tela as datas
print('Tipo da variável=', type(date_ini))
print('Tipo da variável=', type(date_end))
print('Data Inicial=', date_ini)
print('Data Final=', date_end)

In [ ]:
# inicializa os arrays de latitude e longitude
lats_event, lons_event = np.array([]), np.array([])
lats_group, lons_group = np.array([]), np.array([])
lats_flash, lons_flash = np.array([]), np.array([])

# loop da acumulação de relâmpagos
while (date_ini < date_end):

    # mostra na tela a data de processamento atual
    print('PROCESSANDO DATA....', date_ini)

    # estrutura da data
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # baixando GLM
    file_glm = download_GLM(yyyymmddhhmnss, '/content')

    # leitura GLM
    ds = xr.open_dataset(f'/content/{file_glm}.nc')

    # apenda as lats/lons dos eventos, grupos e flashes
    lats_event = np.append(lats_event, ds['event_lat'])
    lons_event = np.append(lons_event, ds['event_lon'])

    lats_group = np.append(lats_group, ds['group_lat'])
    lons_group = np.append(lons_group, ds['group_lon'])

    lats_flash = np.append(lats_flash, ds['flash_lat'])
    lons_flash = np.append(lons_flash, ds['flash_lon'])

    # fecha o arquivo
    ds.close()

    # incrementa 20s no tempo
    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

##**c) Plota figura**

In [ ]:
# Leitura do arquivo
imagem = xr.open_dataset(file_ir_download)

# Limites da imagem
latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])

# Divide por 100 e transforma para Celsius
imagem = (imagem['Band1']/100.)-273.15

# Inverte a matriz
imagem = np.flipud(imagem)

In [ ]:
%%time
# moldura da figura
fig, ax = uplt.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=True, borders=True, innerborders=False,
          labels=True,
          latlines=20, lonlines=20,
          latlim=(latmin,latmax), lonlim=(lonmin,lonmax),
          title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
          small='20px', large='15px')

# carrega tabela de cores
cpt_ir = loadCPT('/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# plota mapa
map1 = ax.imshow(imagem,
                 cmap=cmap_ir,
                 extent=[lonmin, lonmax, latmin, latmax],
                 levels=uplt.arange(-103.0, 105, 1.0),
                 alpha=0.5, zorder=1)

#----------------------------------------------#
#          Plotas os flashes
#----------------------------------------------#
# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região
df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

# transforma de dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

# plota flashes
ax.scatter(lons_flash_filtered, lats_flash_filtered,
           transform=ccrs.PlateCarree(),
           marker='x', s=0.5, facecolor='black', edgecolor='black',
           linewidth=1, alpha=0.8, zorder=2,
           label=f'Flashes={len(lats_flash_filtered)}')
#----------------------------------------------#

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.8, zorder=2)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.4) #, length=0.99

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=True)

# salva figura
fig.save(f'{dir}output/Fig_1_goes16_flash_{ano}{mes}{dia}_{hor}{min}.jpg', dpi=300, bbox_inches='tight')

# **PARTE 3)**: Combinando imagem de satélite com flashes: Área com recorte

In [ ]:
# define a área de recorte
lonmin_rec, lonmax_rec, latmin_rec, latmax_rec = -42, -35, -30, -25

In [ ]:
# Leitura do arquivo para a área de recorte
imagem = xr.open_dataset(file_ir_download).sel(lon=slice(lonmin_rec, lonmax_rec), lat=slice(latmin_rec, latmax_rec))

# Limites da imagem
latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])

# Divide por 100 e transforma para Celsius
imagem = (imagem['Band1']/100.)-273.15

# Inverte a matriz
imagem = np.flipud(imagem)

In [ ]:
# moldura da figura
fig, ax = uplt.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=True, borders=True, innerborders=False,
          labels=True,
          latlines=2, lonlines=2,
          latlim=(latmin_rec,latmax_rec), lonlim=(lonmin_rec,lonmax_rec),
          title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
          small='20px', large='15px')

# carrega tabela de cores
cpt_ir = loadCPT('/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

# plota mapa
map1 = ax.imshow(imagem,
                 cmap=cmap_ir,
                 extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
                 levels=uplt.arange(-103.0, 105, 1.0),
                 alpha=0.8, zorder=1)

#----------------------------------------------#
#          Plotas os flashes
#----------------------------------------------#
# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região
df_flash_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

# transforma de dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

# plota flashes
ax.scatter(lons_flash_filtered, lats_flash_filtered,
           transform=ccrs.PlateCarree(),
           marker='o', s=20, facecolor='white', edgecolor='black',
           linewidth=1, alpha=0.8, zorder=2,
           label=f'Flashes={len(lats_flash_filtered)}')
#----------------------------------------------#

# plota contornos dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.8, zorder=2)

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.3, length=0.71)

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=True)

# salva figura
fig.save(f'{dir}output/Fig_2_goes16_flash_{ano}{mes}{dia}_{hor}{min}_recorte.jpg', dpi=300, bbox_inches='tight')

# **PARTE 4)**: Faz animação das imagens combinadas de satélite e flashes: Área com recorte

##**a) Define o período de imagens de satélite**

In [ ]:
# data INICIAL
anoi, mesi, diai, hori, mini = 2021, 1, 1, 17, 0 # ano, mês, dia, hora e minuto inicial do período

# data FINAL
anof, mesf, diaf, horf, minf = 2021, 1, 1, 18, 30 # ano, mês, dia, hora e minuto final do período

In [ ]:
# define a área de recorte
lonmin_rec, lonmax_rec, latmin_rec, latmax_rec = -42, -35, -30, -25

##**b) Download das imagens de satélite**

In [ ]:
# Inicia o contador de tempo
start_time = t.time()

# Data desejada
date_in = datetime(anoi, mesi, diai, hori, mini)  # 2022-02-02 00:00:00
date_ini = date_in.strftime('%Y%m%d%H%M') # 20220202

date_en = datetime(anof, mesf, diaf, horf, minf)
date_end = date_en.strftime('%Y%m%d%H%M')

print('.... Processando Data ...:', date_ini, date_end, '\n')

# Download dos dados
files_imagens = []
for file in pd.date_range(date_ini,
                          date_end,
                          freq='10min'):

    # Extrai ano e mes
    data = file.strftime('%Y%m%d%H%M')
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')
    hor = file.strftime('%H')
    min = file.strftime('%M')

    # baixa imagem de satélite
    file_ir_download = download_ABI_CPTEC(ano, mes, dia, hor, min, canal)

    # salva o nomes das imagens baixadas
    files_imagens.append(file_ir_download)

# Contabiliza o tempo para baixar os dados
print('\nTempo de download dos dados:', round((t.time() - start_time),2), 'segundos.')

In [ ]:
# mostra os nomes das imagens de satélite que foram baixadas
files_imagens

##**c) Plotando as imagens de satélite + relâmpagos**

In [ ]:
# Loop nas imagens baixadas
for file in files_imagens:

    # extrai a data e horário da imagem
    basename = os.path.basename(os.path.splitext(file)[0]) # S10635334_202101011700.nc
    ano, mes, dia, hor, min = basename[10:14], basename[14:16], basename[16:18], basename[18:20], basename[20:22]

    print('---------------------------------------------------------------------')
    print('PROCESSANDO IMAGEM=', basename)
    print('---------------------------------------------------------------------')

    # Leitura do arquivo
    imagem = xr.open_dataset(file).sel(lon=slice(lonmin_rec, lonmax_rec), lat=slice(latmin_rec, latmax_rec))

    # Divide por 100 e transforma para Celsius
    imagem = (imagem['Band1']/100.)-273.15

    # Inverte a matriz
    imagem = np.flipud(imagem)

    # moldura da figura
    fig, ax = uplt.subplots(axwidth=5, axheight=5, tight=True, proj='pcarree')

    # define formato da figura
    ax.format(coast=False, borders=False, innerborders=False,
              labels=True,
              latlines=2, lonlines=2,
              latlim=(latmin,latmax), lonlim=(lonmin,lonmax),
              title=f'Temperatura (ABI, 10.35 µm) + Flashes (GLM): {ano}-{mes}-{dia} às {hor}:{min} UTC',
              small='20px', large='15px')

    # carrega tabela de cores
    cpt_ir = loadCPT('/content/ir.cpt')
    cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

    # plota mapa
    map1 = ax.imshow(imagem,
                     cmap=cmap_ir,
                     extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
                     levels=uplt.arange(-103.0, 105, 1.0),
                     alpha=0.8, zorder=1)

    # baixa dados do GLM
    lats_event, lons_event, lats_group, lons_group, lats_flash, lons_flash = download_e_acumula_GLM(ano, mes, dia, hor, min)

    #----------------------------------------------#
    #          Plotas os flashes
    #----------------------------------------------#
    # coloca os flashes num dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # seleciona os flashes da região
    df_flash_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

    # transforma de dataframe para array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    # plota flashes
    ax.scatter(lons_flash_filtered, lats_flash_filtered,
               transform=ccrs.PlateCarree(),
               marker='o', s=20, facecolor='white', edgecolor='black',
               linewidth=1, alpha=0.8, zorder=2,
               label=f'Flashes={len(lats_flash_filtered)}')
    #----------------------------------------------#

    # plota contornos dos Estados
    shapefile = list(shpreader.Reader('https://github.com/evmpython/CAT010_UNIFEI_2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.8, zorder=2)

    # plota barra de cores da figura
    fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=13, labelsize=15, space=0.3, length=0.71)

    # adiciona legenda
    ax.legend(loc='lr', ncols=1, frameon=True)

    # salva figura
    fig.save(f'{dir}output/Fig_3_goes16_flash_{ano}{mes}{dia}_{hor}{min}_recorte.jpg', dpi=300, bbox_inches='tight')

##**Faz animação**

In [ ]:
# Images we want to include in the GIF
files = sorted(glob.glob(f'{dir}output/Fig_3_goes16_flash_*.jpg'))
files

In [ ]:
%%time
# cria o GIF
images = []
for file in files:
    images.append(imageio.imread(file))

# salva o GIF
imageio.mimsave(f'{dir}output/Fig_3_animation.gif',
                images,
                duration=500,
                loop=0)

# mostrando a animação na tela
print("\nAbrindo o GIF..\n")

from IPython.display import Image

Image(open(f'{dir}output/Fig_3_animation.gif','rb').read(), width=600)

# **PARTE 5)**: Combinando imagem de satélite com eventos, grupos e flashes: Área com recorte

##**a) Definindo data/horário e canal da imagem**

In [ ]:
# define data/horário e canal
ano, mes, dia, hor, min, canal = '2021', '01', '01', '17', '00', 'ch13'

In [ ]:
# baixa imagem de satélite do FTP do CPTEC
file_ir_download = download_ABI_CPTEC(ano, mes, dia, hor, min, canal)

##**b) Acumula os flashes**

In [ ]:
# define data a INICIAL para acumular os relâmpagos
date_ini = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)))

# define data FINAL para acumular os relâmpagos
date_end = str(datetime(int(ano), int(mes), int(dia), int(hor), int(min)) + timedelta(minutes=10)) # soma 10 minutos na data inicial

In [ ]:
# mostra na tela as datas
print('Tipo da variável=', type(date_ini))
print('Tipo da variável=', type(date_end))
print('Data Inicial=', date_ini)
print('Data Final=', date_end)

In [ ]:
# inicializa os arrays de latitude e longitude
lats_event, lons_event = np.array([]), np.array([])
lats_group, lons_group = np.array([]), np.array([])
lats_flash, lons_flash = np.array([]), np.array([])

# loop da acumulação de relâmpagos
while (date_ini < date_end):

    # mostra na tela a data de processamento atual
    print('PROCESSANDO DATA....', date_ini)

    # estrutura da data
    yyyymmddhhmnss = datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # baixando GLM
    file_glm = download_GLM(yyyymmddhhmnss, '/content')

    # leitura GLM
    ds = xr.open_dataset(f'/content/{file_glm}.nc')

    # apenda as lats/lons dos eventos, grupos e flashes
    lats_event = np.append(lats_event, ds['event_lat'])
    lons_event = np.append(lons_event, ds['event_lon'])

    lats_group = np.append(lats_group, ds['group_lat'])
    lons_group = np.append(lons_group, ds['group_lon'])

    lats_flash = np.append(lats_flash, ds['flash_lat'])
    lons_flash = np.append(lons_flash, ds['flash_lon'])

    # fecha o arquivo
    ds.close()

    # incrementa 20s no tempo
    date_ini = str(datetime.strptime(date_ini, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    print('\n')

##**c) Plota imagem**

In [ ]:
# Leitura do arquivo para a área de recorte
imagem = xr.open_dataset(file_ir_download).sel(lon=slice(lonmin_rec, lonmax_rec), lat=slice(latmin_rec, latmax_rec))

# Limites da imagem
latmin, latmax, lonmin, lonmax = float(imagem['lat'][0]), float(imagem['lat'][-1]), float(imagem['lon'][0]), float(imagem['lon'][-1])

# Divide por 100 e transforma para Celsius
imagem = (imagem['Band1']/100.)-273.15

# Inverte a matriz
imagem = np.flipud(imagem)

In [ ]:
# define a moldura da figura
fig, ax = uplt.subplots([[1, 1, 2, 2],[0, 3, 3, 0]], axwidth=3, axheight=3, tight=True, proj='pcarree')

# formatação dos eixos
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=2, lonlines=2,
          suptitle=f'GLM (GOES-16): {ano}-{mes}-{dia} às {hor}{min} UTC',
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          abc=True, abcstyle='a)', abcsize=5,
          small='15px', large='15px')

# carrega tabela de cores
cpt_ir = loadCPT('/content/ir.cpt')
cmap_ir = cm.colors.LinearSegmentedColormap('cpt_ir', cpt_ir)

#--------------------------------#
#    FIG 1: PLOTA EVENTOS
#--------------------------------#
# plota mapa
map1 = ax[0].imshow(imagem,
                    cmap=cmap_ir,
                    extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
                    levels=uplt.arange(-103.0, 105, 1.0),
                    alpha=0.8, zorder=1)

# Plota os Eventos
#----------------------------------------------#
# coloca os eventos num dataframe
data_event = {'lat': lats_event, 'lon': lons_event}
df = pd.DataFrame(data_event)

# seleciona os eventos da região
df_event_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

# transforma de dataframe para array
lons_event_filtered, lats_event_filtered = df_event_filtered['lon'].values, df_event_filtered['lat'].values

# plota eventos
ax[0].scatter(lons_event_filtered, lats_event_filtered,
              transform=ccrs.PlateCarree(),
              marker='o', s=10, facecolor='white', edgecolor='black',
              linewidth=1, alpha=0.8, zorder=2,
              label=f'Eventos={len(lats_event_filtered)}')
#----------------------------------------------#

# define título da figura
ax[0].format(title='Eventos')

#--------------------------------#
#    FIG 2: PLOTA GRUPOS
#--------------------------------#
# plota mapa
ax[1].imshow(imagem,
             cmap=cmap_ir,
             extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
             levels=uplt.arange(-103.0, 105, 1.0),
             alpha=0.8, zorder=1)

# Plota os Grupos
#----------------------------------------------#
# coloca os grupos num dataframe
data_group = {'lat': lats_group, 'lon': lons_group}
df = pd.DataFrame(data_group)

# seleciona os grupos da região
df_group_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

# transforma de dataframe para array
lons_group_filtered, lats_group_filtered = df_group_filtered['lon'].values, df_group_filtered['lat'].values

# plota grupos
ax[1].scatter(lons_group_filtered, lats_group_filtered,
              transform=ccrs.PlateCarree(),
              marker='o', s=10, facecolor='white', edgecolor='black',
              linewidth=1, alpha=0.8, zorder=2,
              label=f'Grupos={len(lats_group_filtered)}')
#----------------------------------------------#

# define título da figura
ax[1].format(title='Grupos')

#--------------------------------#
#    FIG 3: PLOTA FLASHES
#--------------------------------#
ax[2].imshow(imagem,
             cmap=cmap_ir,
             extent=[lonmin_rec, lonmax_rec, latmin_rec, latmax_rec],
             levels=uplt.arange(-103.0, 105, 1.0),
             alpha=0.8, zorder=1)

# Plota os Flashes
#----------------------------------------------#
# coloca os flashes num dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# seleciona os flashes da região
df_flash_filtered = df[ (df['lon'] > lonmin_rec) & (df['lon'] < lonmax_rec) & (df['lat'] > latmin_rec) & (df['lat'] < latmax_rec)]

# transforma de dataframe para array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

# plota eventos
ax[2].scatter(lons_flash_filtered, lats_flash_filtered,
              transform=ccrs.PlateCarree(),
              marker='o', s=10, facecolor='white', edgecolor='black',
              linewidth=1, alpha=0.8, zorder=2,
              label=f'Flashes={len(lats_flash_filtered)}')
#----------------------------------------------#

# define título da figura
ax[2].format(title='Flashes')
#--------------------------------#

# plota barra de cores da figura
fig.colorbar(map1, loc='r', label='Temperatura de Brilho ($\degree$C)', ticks=25, ticklabelsize=12, labelsize=12, space=0.3, length=0.73)

# adiciona legenda
ax.legend(loc='lr', ncols=1, frameon=True)

# salva figura
fig.save(f'{dir}output/Fig_4_goes16_evento_grupo_flash_{ano}{mes}{dia}_{hor}{min}_recorte.jpg', dpi=300, bbox_inches='tight')